In [3]:
from ipyfilechooser import FileChooser
import os 

def print_file(chooser):
    if not dirModel.selected == None and not dirImage.selected == None: 
        print(dirModel.selected)
        print(dirImage.selected)

#Model Directory
dirModel = FileChooser('')
dirModel.title = '<b>Select Input Model</b>'
dirModel.register_callback(print_file)

#Image Directory
dirImage = FileChooser('')
dirImage.default_path = os.getcwd()
dirImage.title = '<b>Select Input Image</b>'
dirImage.register_callback(print_file)

display(dirModel)
display(dirImage)


FileChooser(path='C:\Users\Jesse B\Documents\Projects\OBJ2CNN', filename='', title='<b>Select Input Model</b>'…

FileChooser(path='C:\Users\Jesse B\Documents\Projects\OBJ2CNN', filename='', title='<b>Select Input Image</b>'…

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.models import Sequential

class_names = ['A-10A_Thunderbolt_II_(Late)', 'A-26C-45_Invader', 'AH-1G', 'B-29A-BN_Superfortress', 'F-100D_Super_Sabre', 'F-104C_Starfighter', 'F4F-4_Wildcat', 'F4U-4B_Corsair', 'F6F-5_Hellcat', 'P-47D-25_Thunderbolt', 'PBJ-1H', 'UH-1B']

model = tf.keras.models.load_model(filepath=dirModel.selected_path)
imgSize = 128

img = tf.keras.utils.load_img(
   dirImage.selected, target_size=(imgSize, imgSize),color_mode='rgba'
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)
print('')
#print(predictions)
for px, plane in enumerate(class_names) :
   print(plane + ' : ' + str(predictions[0][px]))


1/1 [==============================] - 0s 85ms/step
This image most likely belongs to F-100D_Super_Sabre with a 100.00 percent confidence.

A-10A_Thunderbolt_II_(Late) : -6.728912
A-26C-45_Invader : -17.986094
AH-1G : -13.148289
B-29A-BN_Superfortress : 20.67578
F-100D_Super_Sabre : 31.357422
F-104C_Starfighter : 14.727794
F4F-4_Wildcat : -14.803584
F4U-4B_Corsair : -25.558834
F6F-5_Hellcat : -13.007056
P-47D-25_Thunderbolt : -4.0148845
PBJ-1H : -3.401431
UH-1B : -33.66004


In [5]:
#import tensorflow_hub as hub
import cv2
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

width = 128
height = 128

#Load image by Opencv2
img = cv2.imread(dirImage.selected)
#Resize to respect the input_shape
inp = cv2.resize(img, (width , height ))

#Convert img to RGB
rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

# COnverting to uint8
rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

#Add dims to rgb_tensor
rgb_tensor = tf.expand_dims(rgb_tensor , 0)

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [ ]:
#import tensorflow_hub as hub
import pandas as pd

# Loading model directly from TensorFlow Hub
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")

# Loading csv with labels of classes
labels = pd.read_csv('labels.csv', sep=';', index_col='ID')
labels = labels['OBJECT (2017 REL.)']

In [ ]:
# Creating prediction
boxes, scores, classes, num_detections = detector(rgb_tensor)

# Processing outputs
pred_labels = classes.numpy().astype('int')[0] 
pred_labels = [labels[i] for i in pred_labels]
pred_boxes = boxes.numpy()[0].astype('int')
pred_scores = scores.numpy()[0]

# Putting the boxes and labels on the image
for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
    if score < 0.5:
        continue

    score_txt = f'{100 * round(score)}%'
    img_boxes = cv2.rectangle(rgb,(xmin, ymax),(xmax, ymin),(0,255,0),2)      
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img_boxes, label,(xmin, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
    cv2.putText(img_boxes,score_txt,(xmax, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)